In [ ]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
import sys
from typing import List
import numpy as np
import joblib
from pprint import pprint
from math import ceil
import imodelsx.util
import sasc.viz
from copy import deepcopy
from sklearn.cluster import AgglomerativeClustering
from PIL import Image
from scipy.sparse import csr_matrix
from sasc.config import CACHE_DIR, RESULTS_DIR, cache_ngrams_dir, regions_idxs_dir, FMRI_DIR, SAVE_DIR_FMRI
import sasc.modules.fmri_module
from neuro.config import repo_dir, PROCESSED_DIR
from collections import defaultdict
from scipy.stats import norm
from statsmodels.stats.multitest import multipletests
import gemv
from neuro.flatmaps_helper import load_flatmaps

subject = 'S02'
k_values = np.unique(np.logspace(0.5, 3.5, 20, dtype=int))
SUBJECT_VOX_COUNTS_DICT = {'S02': 94251,}

# load connectivity stuff
data = np.load(
    join(FMRI_DIR, 'brain_tune/voxel_neighbors_and_pcs/', 'connectivity.npz'))
connectivity = csr_matrix(
    (data['data'], data['indices'], data['indptr']), shape=data['shape'])

In [88]:
# flatmaps qa dict
settings = ['individual_gpt4_pc']
flatmaps_qa_list = defaultdict(list)
for setting in settings:
    flatmaps_qa_dict = joblib.load(
        join(PROCESSED_DIR, subject.replace('UT', ''), setting + '.pkl'))
    for q in flatmaps_qa_dict.keys():
        flatmaps_qa_list[q].append(flatmaps_qa_dict[q])
flatmaps_qa_dict = {
    q: np.mean(flatmaps_qa_list[q], axis=0)
    for q in flatmaps_qa_list.keys()}

In [89]:
# q = 'Does the input contain a number?'
q = 'Does the input contain a measurement?'
flatmap_q = flatmaps_qa_dict[q].reshape(-1, 1)

### run clustering

In [ ]:
cluster_assignments_dict = {}
for num_clusters in tqdm(k_values):
    out_fname = f'clusters/cluster_aggregated_{num_clusters}.png'
    clustering = AgglomerativeClustering(
        n_clusters=num_clusters,
        connectivity=connectivity,
        linkage='ward',
    )

    clustering.fit(flatmap_q)
    cluster_assignments = clustering.labels_
    cluster_assignments_dict[num_clusters] = deepcopy(cluster_assignments)

    # visualize clusters
    # for i in range(min(20, num_clusters)):
    #     flatmap = (cluster_assignments == i).astype(int)

    #     sasc.viz._save_flatmap(
    #         flatmap, 'S02', fname_save=f'clusters/{i+1}.png')

    # # # read all plots and save as subplots on the same page
    # C = 10
    # R = ceil(min(20, num_clusters)/C)
    # fig, axs = plt.subplots(R, C, figsize=(C * 4, R * 2))
    # axs = axs.ravel()
    # for i in range(min(20, num_clusters)):
    #     axs[i].imshow(Image.open(f'clusters/{i+1}.png'))
    #     axs[i].axis('off')
    #     axs[i].set_title(f'{i + 1}')
    # plt.savefig(out_fname)
joblib.dump(cluster_assignments_dict, join(
    cache_ngrams_dir, f'cluster_assignments_dict_{subject}.jbl'))

# sasc.viz._save_flatmap(
#     flatmap_q.flatten(), 'S02', fname_save=f'clusters/raw_data.png')

### Evaluate clustering

In [69]:
cluster_assignments_dict = joblib.load(join(
    cache_ngrams_dir, f'cluster_assignments_dict_{subject}.jbl'))

In [70]:
cluster_assignments_dict
r = defaultdict(list)
for k in cluster_assignments_dict.keys():
    clusters = cluster_assignments_dict[k]
    cluster_sizes = np.unique(clusters, return_counts=True)[1]
    cluster_vals = np.array([
        np.median(flatmap_q[clusters == i]) for i in range(k)])
    r['num_clusters'].append(k)
    r['cluster_sizes'].append(cluster_sizes)
    r['cluster_driving_scores'].append(cluster_vals)
r = pd.DataFrame(r)

In [ ]:
i_cluster = 15
row = r.iloc[i_cluster]
plt.plot(row['cluster_sizes'], row['cluster_driving_scores'], '.')
plt.grid()
plt.xscale('log')
plt.xlabel('Cluster size')
plt.ylabel('Cluster driving score')
plt.show()

In [ ]:
good_clusters = (row['cluster_sizes'] > 10) & (
    row['cluster_driving_scores'] > 0.01)
good_cluster_idxs = np.arange(len(row['cluster_sizes']))[good_clusters]
k_cluster = row['num_clusters']
num_clusters = len(good_cluster_idxs)
cluster_assignments = cluster_assignments_dict[k_cluster]

# for i in tqdm(range(min(20, num_clusters))):
#     k = good_cluster_idxs[i]
#     flatmap = (cluster_assignments == k).astype(int)

#     sasc.viz._save_flatmap(
#         flatmap, 'S02', fname_save=f'good_clusters/{i+1}.png')

# # read all plots and save as subplots on the same page
# C = 10
# R = ceil(min(20, num_clusters)/C)
# fig, axs = plt.subplots(R, C, figsize=(C * 4, R * 2))
# axs = axs.ravel()
# for i in range(min(20, num_clusters)):
#     axs[i].imshow(Image.open(f'good_clusters/{i+1}.png'))
#     axs[i].axis('off')
#     axs[i].set_title(f'{i + 1}')
# plt.savefig('good_clusters/cluster_aggregated.png')

### Evaluate on gemv

In [76]:
# gemv_flatmaps_dict = gemv_flatmaps_pilot | gemv_flatmaps_pilot5
qa_questions_list, gemv_questions_list = gemv.get_matched_lists()
qa_to_gemv_questions_list = {qa_q: gemv_q for gemv_q, qa_q in zip(
    gemv_questions_list, qa_questions_list)}

In [77]:
normalize_flatmaps = False
gemv_flatmaps_dict_S02, gemv_flatmaps_dict_S03, gemv_flatmaps_dict_S02_timecourse, gemv_flatmaps_dict_S03_timecourse = load_flatmaps(
    normalize_flatmaps, load_timecourse=True)

In [78]:
gemv_flatmaps_dict = gemv_flatmaps_dict_S02

In [ ]:
q_gemv = qa_to_gemv_questions_list[q]
flatmap_gemv = gemv_flatmaps_dict[q_gemv]

# evaluate cluster driving scores
driving_scores = []
qa_driving_scores = []
for k in good_cluster_idxs:
    cluster_driving_score = np.mean(flatmap_gemv[cluster_assignments == k])
    driving_scores.append(cluster_driving_score)
    qa_driving_score = np.mean(flatmap_q[cluster_assignments == k])
    qa_driving_scores.append(qa_driving_score)
# plt.hist(driving_scores)
print('mean gemv driving score', np.mean(driving_scores))
df = pd.DataFrame({'qa_driving_scores': qa_driving_scores,
                   'driving_scores': driving_scores, 'cluster_size': row['cluster_sizes'][good_clusters]})
# plt.plot(qa_driving_scores, driving_scores, '.')
# plt.xlabel('QA driving score')
# plt.ylabel('GEMV driving score')
sns.scatterplot(data=df, x='qa_driving_scores',
                y='driving_scores', hue='cluster_size')
plt.axhline(0, color='k', linestyle='--')
plt.axhline(np.mean(driving_scores), color='pink', linestyle='--')
plt.show()

In [80]:
# t-test on cluster driving score
def zscore_to_pvalue_right_tailed(z):
    """Return the right-tailed p-value for a given z-score."""
    return 1 - norm.cdf(z)

In [81]:
driving_scores = []
zscores = []
pvals = []
for k in good_cluster_idxs:
    cluster_mask = cluster_assignments == k
    timecourse_roi = gemv_flatmaps_dict_S02_timecourse[q_gemv][cluster_mask]
    flatmap_gemv = gemv_flatmaps_dict[q_gemv]

    driving_score = np.mean(flatmap_gemv[cluster_mask])
    stderr = timecourse_roi.mean(
        axis=0).std() / np.sqrt(timecourse_roi.shape[1])
    zscore = driving_score / stderr
    pval = zscore_to_pvalue_right_tailed(zscore)

    driving_scores.append(driving_score)
    zscores.append(zscore)
    pvals.append(pval)

In [ ]:
driving_scores

In [ ]:
sorted(pvals)